In [4]:
import warnings

warnings.filterwarnings("ignore")

import parmed as pmd
from pathlib import Path
import foyer
import json

from parmed.exceptions import MoleculeError, FormatNotFound
from foyer.exceptions import MissingParametersError

/home/chris/miniforge3/envs/xmls/lib/python3.12/site-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/home/chris/miniforge3/envs/xmls/lib/python3.12/site-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/home/chris/miniforge3/envs/xmls/lib/python3.12/site-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.

# OPLS

In [5]:
foyer_opls = foyer.forcefield.Forcefield(name="oplsaa")

opls_dir = Path("OPLS/")
opls_atom_types = dict()
opls_bond_types = dict()
opls_angle_types = dict()
opls_dihedral_types = dict()
opls_improper_types = dict()
error_files = []

In [6]:
for top_file in opls_dir.glob("*.top"):
    try:
        struc = pmd.load_file(filename=str(top_file))
        for atom in struc.atoms:
            opls_atom_types[atom.atom_type.name] = dict(
                sigma=atom.atom_type.sigma / 10,
                epsilon=atom.atom_type.epsilon,
                charge=atom.atom_type.charge,
            )
        for bond in struc.bonds:
            bond_type = f"{bond.atom1.atom_type}-{bond.atom2.atom_type}"
            opls_bond_types[bond_type] = dict(req=bond.type.req / 10, k=bond.type.k)
        for angle in struc.angles:
            angle_type = f"{angle.atom1.atom_type}-{angle.atom2.atom_type}-{angle.atom3.atom_type}"
            opls_angle_types[angle_type] = dict(
                thetaeq=angle.type.theteq, k=angle.type.k
            )
        for dih in struc.dihedrals:
            dihedral_type = f"{dih.atom1.atom_type}-{dih.atom2.atom_type}-{dih.atom3.atom_type}-{dih.atom4.atom_type}"
            opls_dihedral_types[dihedral_type] = dict(
                k=dih.type.phi_k, phase=dih.type.phase
            )
    except (MoleculeError, ValueError, FormatNotFound) as e:
        print(f"Error: {e}")
        error_files.append(str(top_file))

Error: Cannot exclude an atom from itself! Atoms are: <Atom N [5]; In MOL 5> <Atom N [5]; In MOL 5>
Error: Cannot exclude an atom from itself! Atoms are: <Atom N [12]; In MOL 0> <Atom N [12]; In MOL 0>
Error: Cannot exclude an atom from itself! Atoms are: <Atom N [6]; In MOL 6> <Atom N [6]; In MOL 6>
Error: invalid literal for int() with base 10: 'formamide'
Error: Cannot exclude an atom from itself! Atoms are: <Atom N [8]; In MOL 0> <Atom N [8]; In MOL 0>
Error: Could not identify file format
Error: Cannot exclude an atom from itself! Atoms are: <Atom N [14]; In MOL 14> <Atom N [14]; In MOL 14>


In [7]:
new_atom_types = dict()
new_bond_types = dict()
new_angle_types = dict()

for atom_type in opls_atom_types:
    if atom_type not in foyer_opls.atomTypeDefinitions.keys():
        new_atom_types[atom_type] = opls_atom_types[atom_type]

for bond_type in opls_bond_types:
    atom1 = bond_type.split("-")[0]
    atom2 = bond_type.split("-")[1]
    class1 = foyer_opls.atomTypeClasses[atom1]
    class2 = foyer_opls.atomTypeClasses[atom2]
    try:
        bond_params = foyer_opls.get_parameters(
            group="harmonic_bonds", key=[atom1, atom2]
        )
    except MissingParametersError:
        try:
            bond_params = foyer_opls.get_parameters(
                group="harmonic_bonds", key=[class1, class2]
            )
        except MissingParametersError:
            new_bond_types[bond_type] = opls_bond_types[bond_type]

for angle_type in opls_angle_types:
    atom1 = angle_type.split("-")[0]
    atom2 = angle_type.split("-")[1]
    atom3 = angle_type.split("-")[2]
    class1 = foyer_opls.atomTypeClasses[atom1]
    class2 = foyer_opls.atomTypeClasses[atom2]
    class3 = foyer_opls.atomTypeClasses[atom3]
    try:
        angle_params = foyer_opls.get_parameters(
            group="harmonic_angles", key=[atom1, atom2, atom3]
        )
    except MissingParametersError:
        try:
            angle_params = foyer_opls.get_parameters(
                group="harmonic_angles", key=[class1, class2, class3]
            )
        except MissingParametersError:
            new_angle_types[angle_type] = opls_angle_types[angle_type]

# for dih_type in opls_dihedral_types:
#    atom1 = dih_type.split("-")[0]
#    atom2 = dih_type.split("-")[1]
#    atom3 = dih_type.split("-")[2]
#    atom4 = dih_type.split("-")[3]
#    class1 = foyer_opls.atomTypeClasses[atom1]
#    class2 = foyer_opls.atomTypeClasses[atom2]
#    class3 = foyer_opls.atomTypeClasses[atom3]
#    class4 = foyer_opls.atomTypeClasses[atom4]
#    try:
#        dih_params = foyer_opls.get_parameters(group="harmonic_angles", key=[atom1, atom2, atom3])
#    except MissingParametersError:
#        try:
#            angle_params = foyer_opls.get_parameters(group="harmonic_angles", key=[class1, class2, class3])
#        except MissingParametersError:
#            new_angle_types[angle_type] = opls_angle_types[angle_type]

In [9]:
with open("new_atom_types.json", "w") as d:
    json.dump(new_atom_types, d)